# MLB Pitchers Multiple Linear Regression - Statsmodels

In [1]:
# import required modules
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import statsmodels.api as sm

In [2]:
# load dataset
data = pd.read_csv('../../Capstone/Data/MLB_Ppipeline1.csv', header=0)
data.shape

(31710, 42)

In [3]:
data.isna().mean().round(4)*100

age                       0.00
mlb_ID                    0.00
year_ID                   0.00
stint_ID                  0.00
G                         0.00
GS                        0.00
IPouts                    0.00
IPouts_start              0.11
IPouts_relief             0.11
RA                        0.00
xRA                       0.00
xRA_sprp_adj              0.23
xRA_def_pitcher           0.00
PPF                       0.00
PPF_custom                0.11
xRA_final                 0.23
BIP                       0.00
BIP_perc                  0.00
RS_def_total              0.00
runs_above_avg            0.23
runs_above_avg_adj        0.23
runs_above_rep            0.23
RpO_replacement           0.00
GR_leverage_index_avg     0.00
WAR                       0.23
salary                   50.98
teamRpG                   0.00
oppRpG                    0.23
pyth_exponent             0.23
waa_win_perc              0.23
WAA                       0.23
WAA_adj                   0.00
oppRpG_r

In [4]:
data=data.drop(['salary'], axis=1)
data=data.dropna()
data.reset_index(drop=True)

,age,mlb_ID,year_ID,stint_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,...,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,name_common,player_ID,team_ID,lg_ID
0,-1.389691,1.016005,0.533333,-0.291071,-0.750094,-0.691458,-0.879806,-0.660259,-0.436480,-0.844777,...,0.041307,0.298511,0.972452,-0.899199,-0.650090,-0.875375,David Aardsma,aardsda01,SFG,NL
1,-0.919349,1.016005,0.652079,-0.291071,0.984657,-0.691458,-0.265126,-0.660259,1.174129,-0.315958,...,0.526358,0.805143,0.880929,-0.317943,0.170310,-0.150606,David Aardsma,aardsda01,CHC,NL
2,-0.684177,1.016005,0.711452,-0.291071,-0.035785,-0.691458,-0.565206,-0.660259,0.387847,-0.347065,...,0.595545,0.826253,0.692799,-0.508017,-0.522399,-0.491425,David Aardsma,aardsda01,CHW,AL
3,-0.449006,1.016005,0.770825,-0.291071,1.086701,-0.691458,-0.328046,-0.660259,1.009263,-0.098208,...,0.175407,0.425169,0.840252,-0.236661,-0.355122,-0.225782,David Aardsma,aardsda01,BOS,AL
4,-0.213835,1.016005,0.830198,-0.291071,2.413274,-0.691458,0.001073,-0.660259,1.871636,-0.378172,...,0.374732,0.636266,0.870760,0.145818,1.111190,0.057703,David Aardsma,aardsda01,SEA,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30826,-0.919349,1.130544,0.830198,-0.291071,0.168304,-0.691458,-0.584566,-0.660259,0.337120,-0.533707,...,0.397317,0.657375,0.819914,-0.527599,-0.207717,-0.535376,Joel Zumaya,zumayjo01,DET,AL
30827,-0.684177,1.130544,0.889571,-0.291071,0.270348,-0.691458,-0.478086,-0.660259,0.616123,-0.689242,...,-0.363387,-0.187011,0.733476,-0.410105,1.018144,-0.460396,Joel Zumaya,zumayjo01,DET,AL
30828,-0.919349,1.645207,1.186436,-0.291071,-0.648049,-0.604646,-0.768486,-0.624470,-0.246250,-0.906991,...,-0.668138,-0.545875,0.677546,-0.755594,0.935295,-0.790063,Tony Zych,zychto01,SEA,AL
30829,-0.684177,1.645207,1.245809,-0.291071,-0.699071,-0.691458,-0.836246,-0.660259,-0.322342,-0.906991,...,-0.276293,-0.081462,0.824999,-0.828484,0.373325,-0.843559,Tony Zych,zychto01,SEA,AL


In [5]:
features=['age', 'mlb_ID', 'year_ID', 'stint_ID', 'G', 'GS', 'IPouts',
       'IPouts_start', 'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj',
       'xRA_def_pitcher', 'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc',
       'RS_def_total', 'runs_above_avg', 'runs_above_avg_adj',
       'runs_above_rep', 'RpO_replacement', 'GR_leverage_index_avg', 'WAR',
       'teamRpG', 'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA',
       'WAA_adj', 'oppRpG_rep', 'pyth_exponent_rep', 'waa_win_perc_rep',
       'WAR_rep', 'ERA_plus', 'ER_lg']
X=data[features]
y=data.WAR

In [6]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    WAR   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          8.398e+32
Date:                Fri, 19 Jun 2020   Prob (F-statistic):                        0.00
Time:                        01:12:15   Log-Likelihood:                      1.0234e+06
No. Observations:               30831   AIC:                                 -2.047e+06
Df Residuals:                   30794   BIC:                                 -2.046e+06
Df Model:                          37                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
age                    9.414e-17   6.06e-18     15.546      0.000    8.23e-17    1.06e-16
mlb_ID                 4.599e-16   1.01e-17     45.551      0.000     4.4e-16     4.8e-16
year_ID               -1.589e-16   1.17e-17    -13.631      0.000   -1.82e-16   -1.36e-16
stint_ID              -1.108e-16   6.17e-18    -17.964      0.000   -1.23e-16   -9.87e-17
G                     -1.846e-16   1.74e-17    -10.606      0.000   -2.19e-16    -1.5e-16
GS                    -4.835e-16   5.32e-17     -9.089      0.000   -5.88e-16   -3.79e-16
IPouts                 -8.49e-16   2.26e-16     -3.757      0.000   -1.29e-15   -4.06e-16
IPouts_start           1.284e-15   2.07e-16      6.191      0.000    8.77e-16    1.69e-15
IPouts_relief          5.796e-16   7.53e-17      7.694      0.000    4.32e-16    7.27e-16
RA                     3.036e-17   1.06e-16      0.285      0.776   -1.78e-16    2.39e-16
xRA                    1.018e-15   1.42e-16      7.156      0.000    7.39e-16     1.3e-15
xRA_sprp_adj          -3.056e-16   2.84e-17    -10.760      0.000   -3.61e-16    -2.5e-16
xRA_def_pitcher        6.202e-17   1.68e-17      3.697      0.000    2.91e-17    9.49e-17
PPF                    -2.11e-17   1.16e-17     -1.822      0.068   -4.38e-17     1.6e-18
PPF_custom            -2.004e-18   1.13e-17     -0.177      0.859   -2.41e-17    2.01e-17
xRA_final             -5.911e-16    1.9e-16     -3.105      0.002   -9.64e-16   -2.18e-16
BIP                    3.824e-16   7.14e-17      5.354      0.000    2.42e-16    5.22e-16
BIP_perc               9.376e-17   5.49e-17      1.706      0.088   -1.39e-17    2.01e-16
RS_def_total           2.035e-17   8.13e-18      2.503      0.012    4.42e-18    3.63e-17
runs_above_avg          8.36e-16   1.54e-16      5.420      0.000    5.34e-16    1.14e-15
runs_above_avg_adj     1.683e-16   1.28e-16      1.320      0.187   -8.17e-17    4.18e-16
runs_above_rep        -3.077e-16   1.56e-16     -1.974      0.048   -6.13e-16   -2.23e-18
RpO_replacement        2.121e-16    5.7e-17      3.720      0.000       1e-16    3.24e-16
GR_leverage_index_avg -7.119e-17   6.94e-18    -10.263      0.000   -8.48e-17   -5.76e-17
WAR                       1.0000   1.66e-16   6.02e+15      0.000       1.000       1.000
teamRpG                1.184e-15   2.82e-16      4.195      0.000    6.31e-16    1.74e-15
oppRpG                 4.998e-16   1.08e-16      4.646      0.000    2.89e-16    7.11e-16
pyth_exponent         -3.989e-16   9.72e-17     -4.101      0.000   -5.89e-16   -2.08e-16
waa_win_perc          -2.881e-16   5.98e-17     -4.821      0.000   -4.05e-16   -1.71e-16
WAA                   -3.736e-16    1.6e-16     -2.341      0.019   -6.86e-16   -6.08e-17
WAA_adj       